### Google Stock Price Prediction

#### 1. Fire the System

In [18]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

%matplotlib inline

#### 2. Read Data

In [19]:
# importing train data

dataset_train = pd.read_csv('GOOG.csv')
dataset_train.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,49.813286,51.835709,47.800831,49.982655,49.982655,44871300
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942800
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342800
3,2004-08-24,55.412300,55.591629,51.591621,52.239193,52.239193,15319700
4,2004-08-25,52.284027,53.798351,51.746044,52.802086,52.802086,9232100


In [20]:
dataset_train.shape

(4006, 7)

In [21]:
dataset_train.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [22]:
dataset_train.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [23]:
# selecting features for training and predictions

cols = list(dataset_train)[1:6]

# extract dates (will be used in visualization)

datelist_train = list(dataset_train['Date'])
dataset_train['Date'] = pd.to_datetime(dataset_train['Date'], dayfirst=True)

print(f'Training set shape: {dataset_train.shape}')
print(f'All Timestamps: {len(datelist_train)}')
print(f'Selected Features: {cols}')

Training set shape: (4006, 7)
All Timestamps: 4006
Selected Features: ['Open', 'High', 'Low', 'Close', 'Adj Close']


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14404\3968157361.py:8: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dataset_train['Date'] = pd.to_datetime(dataset_train['Date'], dayfirst=True)


#### 3. Data Pre-processing

In [25]:
# removing all the commas and converting data to matrix shape format

dataset_train = dataset_train[cols].astype(str)

for i in cols:
    for j in range(0, len(dataset_train)):
        dataset_train[i][j] = dataset_train[i][j].replace(',','')

dataset_train = dataset_train.astype(float)

# using multiple features (predictors)
training_set = dataset_train.to_numpy()

print(f'Shape of training set : {training_set.shape}')
training_set

Shape of training set : (4006, 5)


array([[  49.813286,   51.835709,   47.800831,   49.982655,   49.982655],
       [  50.316402,   54.336334,   50.062355,   53.95277 ,   53.95277 ],
       [  55.168217,   56.528118,   54.321388,   54.495735,   54.495735],
       ...,
       [1523.130005, 1535.329956, 1498.      , 1513.640015, 1513.640015],
       [1500.      , 1518.689941, 1486.310059, 1518.      , 1518.      ],
       [1521.619995, 1523.439941, 1498.420044, 1515.550049, 1515.550049]])